## Day 24 Lecture 2 Assignment

In this assignment, we will build our a more complex logistic regression model, this time on both numeric and categorical data. We will use the Chicago traffic crashes dataset loaded below and analyze the model generated for this dataset.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
def missingness_summary(df, print_log=False, sort='none'):
    summary = df.apply(lambda x: x.isna().sum() / x.shape[0])
    
    if print_log == True:
        if sort == 'none':
            print(summary)
        elif sort == 'ascending':
            print(summary.sort_values())
        elif sort == 'descending':
            print(summary.sort_values(ascending=False))
        else:
            print('Invalid value for sort parameter.')
        
    return summary


In [4]:
crash = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/traffic_crashes_chicago.csv')

In [5]:
#crash.head()

First, create a binary response column by modifying the "DAMAGE" column. Consider "OVER \$1500" to be the positive class, and under \$1500 to be the negative class.

In [6]:
# answer goes here

#crash['NEW_DAMAGE'] = (crash['DAMAGE'] == 'OVER $1,500').astype(int)

crash['NEW_DAMAGE'] = np.where(crash['DAMAGE'] == 'OVER $1,500', 1, 0)

In [7]:
crash['NEW_DAMAGE'].value_counts()

1    209921
0    162664
Name: NEW_DAMAGE, dtype: int64

Using the code from Day 21, Lecture 1 as a starting point, devise an appropriate way to address missing values. You have a lot of freedom here; we will proceed by taking the following steps:

- Dropping all columns with more than 5% missing data
- Imputing the median for numeric columns with less than 5% missing data (except for STREET_NO; imputing it in this manner would not make any sense)
- Dropping rows with missing data for categorical columns that have less than 5% missing data

In [8]:
# answer goes here
# crash = crash.drop(labels=[['LANE_CNT','INTERSECTION_RELATED_I','NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I','PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I','DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I']], axis=1, inplace=True) 

filtered = crash.dropna(thresh=crash.shape[0]*0.95,how='all',axis=1)

#cols = ['BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
#       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN']

for col in filtered.columns:
#    filtered[col] = filtered[col].fillna(filtered[col].median())
    if filtered[col].dtypes != 'O': 
        filtered[col].fillna(filtered[col].median(), inplace=True)

c:\users\juss\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [27]:
filtered.dropna(inplace=True)

<ipython-input-27-51919896a2e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.dropna(inplace=True)


In [28]:
missingness_summary(filtered)

RD_NO                            0.0
CRASH_DATE                       0.0
POSTED_SPEED_LIMIT               0.0
TRAFFIC_CONTROL_DEVICE           0.0
DEVICE_CONDITION                 0.0
WEATHER_CONDITION                0.0
LIGHTING_CONDITION               0.0
FIRST_CRASH_TYPE                 0.0
TRAFFICWAY_TYPE                  0.0
ALIGNMENT                        0.0
ROADWAY_SURFACE_COND             0.0
ROAD_DEFECT                      0.0
REPORT_TYPE                      0.0
CRASH_TYPE                       0.0
DAMAGE                           0.0
DATE_POLICE_NOTIFIED             0.0
PRIM_CONTRIBUTORY_CAUSE          0.0
SEC_CONTRIBUTORY_CAUSE           0.0
STREET_NO                        0.0
STREET_DIRECTION                 0.0
STREET_NAME                      0.0
BEAT_OF_OCCURRENCE               0.0
NUM_UNITS                        0.0
MOST_SEVERE_INJURY               0.0
INJURIES_TOTAL                   0.0
INJURIES_FATAL                   0.0
INJURIES_INCAPACITATING          0.0
I

Finally, choose a few numeric and categorical features (2-3 of each) to include in the model. (You can definitely include more than this, but too many features, especially categorical ones, will most likely lead to convergence issues). One hot encode the chosen categorical features, being sure to omit one of the categories (which will serve as a "reference" level) to avoid perfect multicollinearity.

Again, you have a lot of freedom here; we will proceed with the following features, dropping the most commonly occurring category for the two categorical variables ("CLEAR" for weather, "REAR END" for first crash type):
POSTED_SPEED_LIMIT, WEATHER_CONDITION, INJURIES_TOTAL, FIRST_CRASH_TYPE

In [29]:
crash_scenario = filtered.filter(['POSTED_SPEED_LIMIT', 'INJURIES_TOTAL','NEW_DAMAGE'])

In [30]:
# answer goes here

weather = pd.get_dummies(filtered['WEATHER_CONDITION'], drop_first=True)
crash_scenario = pd.concat([crash_scenario, weather], axis=1)


crash_type = pd.get_dummies(filtered['FIRST_CRASH_TYPE'], drop_first=True)
crash_scenario = pd.concat([crash_scenario, crash_type], axis=1)




In [31]:
crash_scenario.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362483 entries, 6 to 372584
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   POSTED_SPEED_LIMIT            362483 non-null  int64  
 1   INJURIES_TOTAL                362483 non-null  float64
 2   NEW_DAMAGE                    362483 non-null  int32  
 3   CLEAR                         362483 non-null  uint8  
 4   CLOUDY/OVERCAST               362483 non-null  uint8  
 5   FOG/SMOKE/HAZE                362483 non-null  uint8  
 6   FREEZING RAIN/DRIZZLE         362483 non-null  uint8  
 7   OTHER                         362483 non-null  uint8  
 8   RAIN                          362483 non-null  uint8  
 9   SEVERE CROSS WIND GATE        362483 non-null  uint8  
 10  SLEET/HAIL                    362483 non-null  uint8  
 11  SNOW                          362483 non-null  uint8  
 12  UNKNOWN                       362483 non-nul

Split the data into train and test, with 80% training and 20% testing. By default, the LR output from statsmodels does not include an intercept terms; add a constant column to the training data so that an intercept term is calculated for the LR model (hint: sm.add_constant() is a useful function to accomplish this).

In [32]:
# answer goes here
from sklearn.model_selection import train_test_split

X = crash_scenario.drop('NEW_DAMAGE', axis=1)
Y = crash_scenario['NEW_DAMAGE']

X_train, x_test, Y_train, y_test = train_test_split(X, Y, test_size=0.20)




In [33]:
import statsmodels.api as sm

X_train_con = sm.add_constant(X_train)

Fit the logistic regression model using the statsmodels package and print out the coefficient summary. Which variables (in particular, which categories of our categorical variables) appear to be the most important, and what effect do they have on the probability of a crash resulting in $1500 or more in damages?

In [34]:
# answer goes here

model1 = sm.Logit(Y_train, X_train_con).fit()
model1.summary()


Optimization terminated successfully.
         Current function value: 0.657865
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             NEW_DAMAGE   No. Observations:               289986
Model:                          Logit   Df Residuals:                   289956
Method:                           MLE   Df Model:                           29
Date:                Thu, 15 Oct 2020   Pseudo R-squ.:                 0.04008
Time:                        15:36:31   Log-Likelihood:            -1.9077e+05
converged:                       True   LL-Null:                   -1.9874e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            0.9866      0.652      1.513      0.130      -0.291       2.264
POSTED_SPEED_LIMIT               0.0108      0.001     18.345      0.000       0.010       0.012
INJURIES_TOTAL                   0.7365      0.012     61.925      0.000       0.713       0.760
CLEAR                           -0.6376      0.652     -0.979      0.328      -1.915       0.639
CLOUDY/OVERCAST                 -0.5529      0.652     -0.848      0.396      -1.831       0.725
FOG/SMOKE/HAZE                  -0.4289      0.658     -0.652      0.514      -1.718       0.860
FREEZING RAIN/DRIZZLE           -0.1339      0.680     -0.197      0.844      -1.466       1.198
OTHER                           -0.3442      0.655     -0.525      0.599      -1.629       0.940
RAIN                            -0.5122      0.652     -0.786      0.432      -1.789       0.765
SEVERE CROSS WIND GATE          -0.1574      0.710     -0.222      0.825      -1.549       1.234
SLEET/HAIL                      -0.5346      0.659     -0.811      0.417      -1.826       0.757
SNOW                            -0.5109      0.652     -0.784      0.433      -1.789       0.767
UNKNOWN                         -0.4666      0.652     -0.716      0.474      -1.744       0.811
ANIMAL                          -1.1042      0.150     -7.344      0.000      -1.399      -0.810
FIXED OBJECT                     0.0510      0.024      2.163      0.031       0.005       0.097
HEAD ON                         -0.0426      0.047     -0.900      0.368      -0.135       0.050
OTHER NONCOLLISION              -1.1407      0.065    -17.506      0.000      -1.268      -1.013
OTHER OBJECT                    -0.6568      0.042    -15.796      0.000      -0.738      -0.575
OVERTURNED                       0.7704      0.251      3.075      0.002       0.279       1.261
PARKED MOTOR VEHICLE            -0.4837      0.015    -31.925      0.000      -0.513      -0.454
PEDALCYCLIST                    -2.4742      0.040    -61.867      0.000      -2.553      -2.396
PEDESTRIAN                      -2.3787      0.032    -74.125      0.000      -2.442      -2.316
REAR END                        -0.6836      0.015    -45.993      0.000      -0.713      -0.654
REAR TO FRONT                   -0.6834      0.067    -10.134      0.000      -0.816      -0.551
REAR TO REAR                    -0.9281      0.138     -6.737      0.000      -1.198      -0.658
REAR TO SIDE                    -0.5686      0.085     -6.727      0.000      -0.734      -0.403
SIDESWIPE OPPOSITE DIRECTION    -0.6305      0.033    -18.892      0.000      -0.696      -0.565
SIDESWIPE SAME DIRECTION        -0.6608      0.016    -41.736      0.000      -0.692      -0.630
TRAIN                            0.7197      0.771      0.934      0.350      -0.791       2.230
TURNING                         -0.1965      0.017    -11.871      0.000      -0.229      -0.164
========================================

Which variables (in particular, which categories of our categorical variables) appear to be the most important, and what effect do they have on the probability of a crash resulting in $1500 or more in damages?

Injury totals and overturning the vehicle leads to a higher probabity of damamges beind more than 1500.

In [35]:
# answer goes here
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(max_iter=1000)
logit.fit(x_test, y_test)


LogisticRegression(max_iter=1000)

In [36]:
logit.score(X_train, Y_train)

0.5832798824770851

In [37]:
logit.score(x_test, y_test)

0.582782735837345

Create a LogisticRegression model with sklearn. Use the .predict() method (using X_test) to get a y_pred. Create a confusion matrix comparing your actual y_test to your prediction. What do you notice about your type of error?

In [39]:
from sklearn.metrics import confusion_matrix

In [41]:
results = logit.predict(x_test)

array([[ 9797, 22113],
       [ 8134, 32453]], dtype=int64)

There are lots of false positives and the number of true postitives is also high. 